In [1]:
#Network Flow Model
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np

m= gp.Model(name="Network")

# Nodes
nodes =[1,2,3,4,5,6,7,8]

# arcs and their capacity, we use 1e10 to represent infinite capacity (more than max supply of 1000+1000)
arc,capacity = gp.multidict({   
    (1,3):1e10,
    (1,4):1e10,
    (1,8):1e10,
    (2,3):1e10,
    (2,4):1e10,
    (2,8):1e10,
    (3,4):25,
    (3,5):1e10,
    (3,6):1e10,
    (3,7):1e10,
    (4,3):25,
    (4,5):1e10,
    (4,6):1e10,
    (4,7):1e10})

# Cost
cost={
        (1,3):7,
        (1,4):8,
        (1,8):0,
        (2,3):4,
        (2,4):7,
        (2,8):0,
        (3,4):0,
        (3,5):25,
        (3,6):5,
        (3,7):17,
        (4,3):0,
        (4,5):29,
        (4,6):8,
        (4,7):5}

# Demand for inflow
demand = {
        1:-1000,
        2:-1000,
        3:0,
        4:0,
        5:450,
        6:500,
        7:610,
        8:440}

# Create variables
flow = m.addVars(arc,obj=cost,name="flow")

# Bounds on the flow
m.addConstrs((flow[i,j] <= capacity[i, j] for i, j in arc),"capacity")
# Equivalent version using Python looping
#for i, j in arc:
 #  m.addConstr(flow[i,j] <= capacity[i,j],"cap[%s, %s]" % (i, j))

# Flow balance constraints
m.addConstrs((flow.sum('*',i)-flow.sum(i,'*') == demand[i] for i in nodes), "node")
# Equivalent version using Python looping
#m.addConstrs((-quicksum(flow[i,j] for i,j in arc.select(i,'*'))+quicksum(flow[i,j] for i,j in arc.select('*',i))==inflow[i]
#for i in nodes), "node")

# Compute optimal solution
m.optimize()

# Print solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', flow)
    for i, j in arc:
        if solution[i, j] > 0:
            print('%s -> %s: %g' % (i, j, solution[i, j]))
        else:
            print('%s <- %s: %g' % (i, j, -solution[i, j]))

Academic license - for non-commercial use only - expires 2022-07-19
Using license file C:\Users\jkettune\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22 rows, 14 columns and 42 nonzeros
Model fingerprint: 0x8efb46f2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 15 rows and 1 columns
Presolve time: 0.01s
Presolved: 7 rows, 13 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6800000e+04   4.143199e+02   0.000000e+00      0s
       5    2.5355000e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.03 seconds
Optimal objective  2.535500000e+04
1 <- 3: -0
1 -> 4: 560
1 